Import necessary libraries

In [412]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_val_predict, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, chi2, RFE,mutual_info_classif
from numpy import mean, std
import matplotlib.pyplot as plt



Import all data

In [395]:
seasons = range(2010,2023)
team_seasons = range(2009,2023)
premier_match_data = {}
premier_team_data = {}
premier_player_data = {}
championship_match_data = {}
championship_team_data = {}
merge_premier={}
merge_championship={}
all_match_data = {}

for i in team_seasons:
    team_data_file = f'..//data//england_premier_team//england-premier-league-teams-{i}-to-{i+1}-stats.csv'
    championship_team_data_file = f'..//data//england_championship_team//england-championship-teams-{i}-to-{i+1}-stats.csv'
    championship_team_data[f'{i}_{i+1}'] = pd.read_csv(championship_team_data_file)
    premier_team_data[f'{i}_{i+1}'] = pd.read_csv(team_data_file)

for i in seasons:
    match_data_file = f'..//data//england_premier_match//england-premier-league-matches-{i}-to-{i+1}-stats.csv'
    player_data_file = f'..//data//england_premier_player//england-premier-league-players-{i}-to-{i+1}-stats.csv'
    championship_match_data_file =  f'..//data//england_championship_match//england-championship-matches-{i}-to-{i+1}-stats.csv'
    

    premier_match_data[f'{i}_{i+1}'] = pd.read_csv(match_data_file)
    premier_player_data[f'{i}_{i+1}'] = pd.read_csv(player_data_file)
    championship_match_data[f'{i}_{i+1}'] = pd.read_csv(championship_match_data_file)
    
    merge_premier[f'{i}_{i+1}'] = premier_match_data[f'{i}_{i+1}'].merge(premier_team_data[f'{i-1}_{i}'], left_on ='home_team_name',right_on = 'common_name',suffixes=('', '_home'))
    merge_championship[f'{i}_{i+1}'] = championship_match_data[f'{i}_{i+1}'].merge(championship_team_data[f'{i-1}_{i}'], left_on ='home_team_name',right_on = 'common_name',suffixes=('', '_home'))

    all_match_data[f'{i}_{i+1}'] = pd.concat([merge_premier[f'{i}_{i+1}'], merge_championship[f'{i}_{i+1}']], ignore_index=True)
    all_match_data[f'{i}_{i+1}'] = all_match_data[f'{i}_{i+1}'].loc[:, ~all_match_data[f'{i}_{i+1}'].columns.duplicated()].copy()

In [396]:
all_match_data['2013_2014'].to_csv("merge.csv", index=False)

Pre processing data
1. fill all missing value with -1 
2. Split goal scoring minutes into first half and second half
3. label encoding all object column
4. group previous three matches as the feature to train model.

In [397]:
def bin_goal_timings(goal_timings):
    # Parse the string into a list of integers
    if not isinstance(goal_timings, str):
        return (0,0)  # Return (0,0) if goal_timings is not a string

    if goal_timings=='-1':
        return (0,0)
    # Split the string into a list of times
    goal_times = goal_timings.split(',')

    first_half_goals = 0
    second_half_goals = 0

    for time in goal_times:
        # Check if it's stoppage time
        if "'" in time:
            time_parts = time.split("'")
            # Consider stoppage time as part of the second half
            if int(time_parts[0]) >= 45:
                second_half_goals += 1
        else:
            # Check if the goal was scored in the first or second half
            if int(time) <= 45:
                first_half_goals += 1
            else:
                second_half_goals += 1
    return first_half_goals, second_half_goals


In [398]:
# Create a list of all team names across all seasons
all_teams = []
all_referee = []
all_stadiums = []
for season in all_match_data.keys():
    all_teams.extend(list(all_match_data[season]['home_team_name'].unique()))
    all_teams.extend(list(all_match_data[season]['away_team_name'].unique()))
    all_referee.extend(list(all_match_data[season]['referee'].unique()))
  
    all_stadiums.extend(list(all_match_data[season]['stadium_name'].unique()))


    

In [399]:
all_match_data['2010_2011']['total_goal_count_home']

0      108
1      108
2      108
3      108
4      108
      ... 
737    115
738    115
739    115
740    115
741    115
Name: total_goal_count_home, Length: 742, dtype: int64

In [400]:
le = LabelEncoder()
le_teams = LabelEncoder()
le_referee = LabelEncoder()
le_stadium = LabelEncoder()

le_teams.fit(all_teams)
le_referee.fit(all_referee)
le_stadium.fit(all_stadiums)


avoid_column = ['home_team_name','away_team_name','referee','stadium_name']
drop_column  = ['team_name','common_name','country','season']
for season, df in all_match_data.items():
    df.drop(drop_column, axis=1)
    # Transform teams
    df['home_team_name'] = le_teams.transform(df['home_team_name'])
    df['away_team_name'] = le_teams.transform(df['away_team_name'])
    df['referee'] = le_referee.transform(df['referee'])
    df['stadium_name'] = le_stadium.transform(df['stadium_name'])

    all_match_data[season] = df.fillna(-1,inplace=True)
    # Convert the column to string type
    for col in df.columns:
        if df[col].dtype == 'object' and col not in avoid_column:
            # Convert the column to string type
            df[col] = df[col].astype(str)
            
            # Apply the label encoder
            le.fit(df[col])
            df[col] = le.transform(df[col])

    
    all_match_data[season] = df
    

Now we need to group previous 3 game stats as features of this game

In [401]:
def calculate_rolling_stats(group, window_size=3):
    # List of columns to skip
    columns_to_skip = ['timestamp', 'date_GMT', 'status', 'home_team_name', 'away_team_name', 'referee', 'Game Week']
    group = group.sort_values('Game Week')
    
    # Shift the data down 1 so the current game's stats aren't included
    group = group.shift(1)
    
    # Iterate over all columns in the group
    for col in group.columns:
        # Skip the column if it is in the list of columns to skip
        if col not in columns_to_skip:
            # The rolling window size is 3, which means the previous 3 games
            group['rolling_avg_' + col] = group[col].rolling(window=window_size).mean()

    # Drop the first two rows (which won't have any rolling data)
    group = group.iloc[window_size:]
    
    return group



Get all rolling data and remove those without. 
Leave the 2022-23 as a test dataset.

In [402]:
# Initialize empty dictionaries to hold the stats for all seasons
all_seasons_home_stats = {}
all_seasons_away_stats = {}

# Get a list of all season keys, sorted in ascending order
seasons = sorted(all_match_data.keys())
window_size = 2
# Use all seasons except the last one
for season in seasons[:-1]:
    # Apply the calculate_rolling_stats function to the home and away data for this season
    all_seasons_home_stats[season] = all_match_data[season].groupby('home_team_name').apply(calculate_rolling_stats,window_size)
    all_seasons_away_stats[season] = all_match_data[season].groupby('away_team_name').apply(calculate_rolling_stats,window_size)

# Convert the dictionaries to dataframes
all_seasons_home_stats_df = pd.concat(all_seasons_home_stats, keys=all_seasons_home_stats.keys())
all_seasons_away_stats_df = pd.concat(all_seasons_away_stats, keys=all_seasons_away_stats.keys())

# Now, the last season's data can be accessed separately like this
last_season_home_stats = all_match_data[seasons[-1]].groupby('home_team_name').apply(calculate_rolling_stats,window_size)
last_season_away_stats = all_match_data[seasons[-1]].groupby('away_team_name').apply(calculate_rolling_stats,window_size)



In [403]:
all_seasons_home_stats_df.to_csv("all_season_home.csv", index=False)
all_seasons_away_stats_df.to_csv("all_season_away.csv", index=False)
last_season_home_stats.to_csv("validation_home_stats.csv", index=False)
last_season_away_stats.to_csv("validation_away_stats.csv", index=False)

Train

In [404]:
home_goals = all_seasons_home_stats_df['home_team_goal_count']
away_goals = all_seasons_away_stats_df['away_team_goal_count']

columns_to_drop_home = ['home_team_goal_count','total_goal_count','total_goals_at_half_time','home_team_goal_count_half_time','away_team_goal_count_half_time','home_team_goal_timings'
                   ,'away_team_goal_timings','home_team_corner_count','away_team_corner_count','home_team_yellow_cards','home_team_red_cards','away_team_yellow_cards','away_team_red_cards',
                   'home_team_first_half_cards','home_team_second_half_cards','away_team_first_half_cards','away_team_second_half_cards','home_team_shots','away_team_shots','home_team_shots_on_target',
                   'away_team_shots_on_target','home_team_shots_off_target','away_team_shots_off_target','home_team_fouls','away_team_fouls','home_team_possession','away_team_possession',
                   'status','attendance']

columns_to_drop_away = ['away_team_goal_count','total_goal_count','total_goals_at_half_time','home_team_goal_count_half_time','away_team_goal_count_half_time','home_team_goal_timings'
                   ,'away_team_goal_timings','home_team_corner_count','away_team_corner_count','home_team_yellow_cards','home_team_red_cards','away_team_yellow_cards','away_team_red_cards',
                   'home_team_first_half_cards','home_team_second_half_cards','away_team_first_half_cards','away_team_second_half_cards','home_team_shots','away_team_shots','home_team_shots_on_target',
                   'away_team_shots_on_target','home_team_shots_off_target','away_team_shots_off_target','home_team_fouls','away_team_fouls','home_team_possession','away_team_possession',
                   'status','attendance']

# Drop these columns from the predictor variables DataFrame
y_home = all_seasons_home_stats_df['home_team_goal_count']
y_away = all_seasons_away_stats_df['away_team_goal_count']
X_home = all_seasons_home_stats_df.drop(columns_to_drop_home, axis=1)
X_away = all_seasons_away_stats_df.drop(columns_to_drop_away, axis=1)
test_home_X = last_season_home_stats.drop(columns_to_drop_home, axis=1)
test_home_y = last_season_home_stats['home_team_goal_count']
test_away_X = last_season_away_stats.drop(columns_to_drop_away, axis=1)
test_away_y = last_season_away_stats['away_team_goal_count']


In [405]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
}
grid_params = {
    "Linear Regression": {
        'fit_intercept': [True, False],  # Specifies if a constant (a.k.a. bias or intercept) should be added to the decision function.
    },
    "Random Forest": {
        'n_estimators': [50, 100, 200, 300],  # Number of trees in the forest
        'max_depth': [None, 5, 10, 20, 30],  # Maximum depth of the tree
        'min_samples_split': [2, 5, 10],  # The minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 2, 4],  # The minimum number of samples required to be at a leaf node
        'bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
    },
    "Gradient Boosting": {
        'n_estimators': [50, 100, 200, 300],  # Number of boosting stages to perform
        'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Learning rate
        'max_depth': [3, 5, 7, 10],  # Maximum depth of the individual regression estimators
        'subsample': [0.5, 0.7, 1.0],  # The fraction of samples to be used for fitting the individual base learners
        'min_samples_split': [2, 5, 10],  # The minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 2, 4],  # The minimum number of samples required to be at a leaf node
    },
}

Select most useful features
1. note! after using mutual info it appears that less the feature selected, the worse the accuracy is

In [406]:
selector = SelectKBest(mutual_info_classif, k=20)
selector.fit_transform(X_home, y_home)

selected_features_home = selector.get_support(indices=True)
print("Selected features_home: ", X_home.columns[selected_features_home])

selector.fit_transform(X_away,y_away)
selected_features_away = selector.get_support(indices=True)
print("Selected features_away: ", X_away.columns[selected_features_away])

Selected features_home:  Index(['home_ppg', 'away_ppg', 'team_a_xg', 'odds_ft_home_team_win',
       'odds_ft_away_team_win', 'performance_rank', 'goal_difference',
       'win_percentage_home', 'rolling_avg_home_ppg',
       'rolling_avg_home_team_goal_count', 'rolling_avg_total_goal_count',
       'rolling_avg_total_goals_at_half_time',
       'rolling_avg_home_team_goal_count_half_time',
       'rolling_avg_home_team_goal_timings',
       'rolling_avg_home_team_shots_on_target',
       'rolling_avg_odds_ft_home_team_win',
       'rolling_avg_odds_ft_away_team_win', 'rolling_avg_stadium_name',
       'rolling_avg_fts_half_time', 'rolling_avg_corners_per_match'],
      dtype='object')
Selected features_away:  Index(['home_ppg', 'away_ppg', 'team_b_xg', 'odds_ft_home_team_win',
       'odds_ft_away_team_win', 'odds_btts_yes', 'stadium_name',
       'goals_scored_home', 'shots_on_target_home', 'over05_percentage',
       'rolling_avg_away_ppg', 'rolling_avg_away_team_goal_count',
      

In [413]:
def find_best_param(X, y, models, cv=5):
    best_models = {}
    mse_scores = {}
    for model_name, model in models.items():
        # clf = GridSearchCV(model, grid_params[model_name], cv=cv, scoring='neg_mean_squared_error')
        clf = RandomizedSearchCV(model, grid_params[model_name], cv=cv, scoring='neg_mean_squared_error', n_iter=100, random_state=42)
        clf.fit(X, y)
        
        print(f"Best parameters for {model_name}:")
        print(clf.best_params_)
        best_models[model_name] = clf.best_estimator_

        # Predict with cross validation and compute MSE
        y_pred = cross_val_predict(clf.best_estimator_, X, y, cv=cv)
        mse = mean_squared_error(y, y_pred)
        mse_scores[model_name] = mse
        print(f'MSE for {model_name}: {mse}')
    
    # Plot
    plt.figure(figsize=(10, 5))
    plt.bar(mse_scores.keys(), mse_scores.values(), color='blue', alpha=0.6)
    plt.ylabel('Mean Squared Error')
    plt.title('Cross-validated MSE for different models')
    plt.show()
    
    return best_models


def fit_and_compute_mse(models, X, y, selected_features = None, k=10):
    mse_scores = {}
    
    X_selected = X if selected_features is None else X[X.columns[selected_features]]
    for name, model in models.items():
        scores = cross_val_score(model, X_selected, y, cv=k, scoring='neg_mean_squared_error')
        # Take the negative of the scores to get the MSE
        mse_scores[name] = -scores.mean()
        print(f"{name} loss is {-scores.mean()}")
    # Plot
    plt.figure(figsize=(10, 5))
    plt.bar(mse_scores.keys(), mse_scores.values(), color='blue', alpha=0.6)
    plt.ylabel('Mean Squared Error')
    plt.title('Cross-validated MSE for different models')
    plt.show()

def test_on_test(models,test_X,test_y):
    mse_scores = {}
    
    for name, model in models.items():
        y_pred = model.predict(test_X)
        scores = mean_squared_error(y_pred=y_pred,y_true=test_y)
        # Take the negative of the scores to get the MSE
        mse_scores[name] = -scores.mean()
        print(f"{name} loss is {-scores.mean()}")
    # Plot
    plt.figure(figsize=(10, 5))
    plt.bar(mse_scores.keys(), mse_scores.values(), color='blue', alpha=0.6)
    plt.ylabel('Mean Squared Error')
    plt.title('MSE for test set')
    plt.show()

In [415]:
home_best_estimator = find_best_param(X_home,y_home,models)
away_best_estimator = find_best_param(X_away,y_away,models)

c:\Users\xu741\miniconda3\envs\py39\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 2 is smaller than n_iter=100. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for Linear Regression:
{'fit_intercept': False}
MSE for Linear Regression: 0.6627860170373007
